# The Cancer Gene Atlas Cancer (TCGA) - Skin Cutaneous Melanoma (SKCM)

Author: Moshe Silverstein <br>
Date: 7-17 <br>
Data Source: https://cancergenome.nih.gov/

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
import seaborn as sns
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

In [3]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [4]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/TCGA/untility_functions.py'>

# Versions Of Modules In Use

In [5]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget, seaborn 

Software versions
Python 3.5.5 64bit [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 6.3.1
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.14.2
pandas 0.22.0
clustergrammer_widget 1.9.0
seaborn 0.8.1
Fri May 11 11:17:26 2018 EDT

# Functions

In [6]:
def getGenes(inputDF):
    
    inputDF.reset_index(inplace=True)
    
    inputDF.rename(columns={'index':'Gene Symbol'}, inplace=True)

    inputDF.set_index('Gene Symbol', inplace=True)

    lst = []

    for index in  inputDF.index:
        lst.append(index.split('|')[5])

    inputDF.index = lst

In [7]:
def getSampleID(inputDF, metaDF):
    
    lst = []
    
    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()

        ID = ('-').join(metaDF.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        
        ID = ID+'-'+metaDF.loc[col, 'a_AliquotBarcode'].split('-')[3][0:2]

        lst.append(ID)

        
    inputDF.columns = lst

In [8]:
def getSampleData(inputDF, metaDF, SampleTypes):
    
    lst = []

    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()
        
#         ID = ('-').join(meta.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        ID = ('-').join(col.split('-')[0:3])
        
        donor = 'Donor:'+ID
        
#         if '01' in str(col.split('-')[3]):
#             sampleType = 'Primary Solid Tumor'
#         elif '11' in str(col.split('-')[3]):
#             sampleType= 'Solid Tissue Normal'
        
        if ID in metaDF.index:
            info = (donor, 
                'Age:'+str(metaDF.loc[ID, 'age_at_initial_pathologic_diagnosis']),
                'Gender:'+str(metaDF.loc[ID, 'gender']),    
                'SampleType:'+SampleTypes[str(col.split('-')[3])],
                'Stage:'+str(metaDF.loc[ID, 'pathologic_stage']),
                'Histology:'+str(metaDF.loc[ID, 'histological_type']),
                'Tissue:'+str(metaDF.loc[ID, 'tumor_tissue_site']), 
                'Tumor:'+str(metaDF.loc[ID, 'tumor_type']),    
                'Grade'+str(metaDF.loc[ID, 'neoplasm_histologic_grade'])
#                 'SampleType:'+str(ID.split('-')[3])
               )
#             for col in metaDF.columns:
#                 if len(metaDF[col].dropna()) >= .5*len(metaDF[col]) and len(metaDF[col].unique()) > 2 and len(metaDF[col].unique()) < len(metaDF[col]) and 'age' not in col:
#                     info = info + (str(col)+':'+str(metaDF.loc[ID, col]),)
        lst.append(info)

        
    inputDF.columns = lst

In [9]:
SampleTypes ={
    '01':'Primary Solid Tumor',
    '02':'Recurrent Solid Tumor',
    '03':'Primary Blood Derived Cancer - Peripheral Blood',
    '04':'Recurrent Blood Derived Cancer - Bone Marrow',
    '05':'Additional - New Primary',
    '06':'Metastatic',
    '07':'Additional Metastatic',
    '08':'Human Tumor Original Cells',
    '09':'Primary Blood Derived Cancer - Bone Marrow',
    '10':'Blood Derived Normal',
    '11':'Solid Tissue Normal',
    '12':'Buccal Cell Normal',
    '13':'EBV Immortalized Normal',
    '14':'Bone Marrow Normal',
    '15':'sample type 15',
    '16':'sample type 16',
    '20':'Control Analyte',
    '40':'Recurrent Blood Derived Cancer - Peripheral Blood',
    '50':'Cell Lines',
    '60':'Primary Xenograft Tissue',
    '61':'Cell Line Derived Xenograft Tissue',
    '99':'sample type 99'
             }

# Path to Output Files

In [13]:
path = '/Users/moshesilverstein/Documents/Harmonizome/TCGA/Output/'

# Load Meta Data

In [10]:
meta = pd.read_csv('Input/TCGA_Metadata.csv', index_col=9)

In [11]:
meta.head()

,a_AliquotBarcode,b_disease,a_objectSizeBytes,b_uploaded,b_state,b_library_type,b_center,b_assembly,a_GCSobject
a_CGHubAnalysisID,,,,,,,,,
3a8e6a74-137f-468f-8987-fa0acdde2836,TCGA-CS-6188-01A-11R-1896-07,LGG,6354301047,8/14/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
c63d44f1-c9ad-4391-b1aa-0feb3713441b,TCGA-EO-A22Y-01A-11R-A180-07,UCEC,5497637066,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
21912089-1e42-4bcc-9ad9-fe9a9b88fb09,TCGA-94-A5I4-01A-11R-A26W-07,LUSC,4600317850,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
6b86e544-0372-434a-9f28-6fa4075dd228,TCGA-N5-A4RV-01A-21R-A28V-07,UCS,4964387768,8/30/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
30518eb4-2783-4988-bcd4-36a1a8bb4dfa,TCGA-BK-A56F-01A-32R-A27V-07,UCEC,5668623460,8/31/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...


In [12]:
meta.shape

(11373, 9)

# Load Sample MetaData

In [14]:
sample_meta = pd.read_csv('Input/metadata_Skin Cutaneous Melanoma [SKCM]', sep='\t', index_col=1)

In [15]:
sample_meta.head()

,Unnamed: 0,Study,Project,ParticipantUUID,TSSCode,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision,batch_number,bcr,clinical_M,...,BMI,age_began_smoking_in_years,h_pylori_infection,other_dx,other_malignancy_anatomic_site,other_malignancy_histological_type,other_malignancy_malignancy_type,stopped_smoking_year,venous_invasion,year_of_tobacco_smoking_onset
ParticipantBarcode,,,,,,,,,,,,,,,,,,,,,
TCGA-GF-A6C9,225,SKCM,TCGA,46801BE1-F035-421F-AAE4-81BAF3BB8D40,GF,78.0,NaN,316,Nationwide Children's Hospital,NaN,...,29.4,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-FW-A3I3,479,SKCM,TCGA,436A0116-EADA-4B8F-AAFE-29EB7F3241C4,FW,59.0,NaN,206,Nationwide Children's Hospital,NaN,...,35.1,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-D3-A8GQ,2549,SKCM,TCGA,320A2DCA-EE07-4D14-A7B8-178636636880,D3,66.0,NaN,388,Nationwide Children's Hospital,NaN,...,27.1,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-D3-A8GS,2550,SKCM,TCGA,588A48F7-947C-463E-8991-9A4EE82683AA,D3,52.0,NaN,388,Nationwide Children's Hospital,NaN,...,33.9,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-D3-A8GJ,2551,SKCM,TCGA,42FB358C-CC9B-4963-9D16-4A7AA915002A,D3,18.0,NaN,388,Nationwide Children's Hospital,NaN,...,23.1,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
sample_meta.shape

(470, 70)

# Load Data

In [17]:
matrix = pd.read_csv('Input/TCGA_SKCM_tpm.tsv', sep='\t', index_col=0)

In [18]:
matrix.head()

,fd85644a-09d9-4b61-993a-2da90f7907b0,56ff6499-0cc1-4108-aeb5-68ad6e371df3,bd7f0212-7a22-4b76-b5ed-f7879bad4c6b,51b4f189-f471-46c8-9a83-45dea4ff3348,91b61c25-e966-4c03-a71b-9be9e39f8465,bc939517-e335-47ca-82f9-98c580f302be,9e74ed10-91aa-44b7-9d04-7acb5d40fd85,0556b68e-a097-4d6f-b337-0554293ffab4,a1fa96bb-dd0e-4d8d-8305-f7f8c342470c,1baad8de-4a9d-4381-8970-02063c295ccc,...,e8967855-1a73-4bf0-9751-5f24d5118173,4b77ade4-dc95-4ac0-bbb3-0923e9cf625c,57d91a5a-589c-49c0-a169-8cbca345c58a,5ca8002d-74d6-42ac-805a-977c842e7740,5f8ba9ee-aafb-4902-b8d5-94d2739eb8fd,ee76b3b0-c6a8-4176-ae7a-7af338cc8ede,d541ac75-ec19-4f69-ba1d-2c28e86e0f49,4726a0f4-1c9e-4978-a4b1-47f822a4df71,ca045c46-1640-4842-9a72-4f847b413907,b46beddd-ea86-480b-a65c-aac06e3568f2
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,145.07500,285.49300,257.100000,114.16200,153.84400,121.991000,166.685000,319.796000,66.73140,207.013000,...,196.522000,117.401000,180.641000,163.23800,234.59100,316.28100,2.138910e+02,108.430000,197.75000,181.38100
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,53.03300,15.87720,14.433500,52.91730,38.91390,52.214600,83.282300,23.645200,44.19540,18.276900,...,29.718100,36.444400,38.519500,24.75210,7.72247,29.86650,1.210080e+02,5.964770,38.61060,11.49890
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,27.30070,15.12670,17.462500,21.00280,12.56990,26.143900,12.155300,9.955220,28.20380,19.578400,...,14.399500,32.773700,9.270470,13.71660,12.12620,21.86140,5.702150e+00,20.502800,15.06870,13.32210
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,80.82990,51.13760,28.742100,127.71100,44.92330,51.669400,39.728500,66.014900,102.14200,42.745300,...,21.251500,105.298000,44.568900,45.29140,19.03680,49.97300,2.522550e+02,9.587440,27.71250,31.32520
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,1.34305,1.08943,0.110161,1.25061,3.21387,0.383467,0.340106,0.214055,2.04426,0.091373,...,0.737908,0.395834,0.394323,3.66791,0.17144,1.10758,3.469740e-07,0.317372,1.97814,1.22536


In [19]:
matrix.shape

(199169, 473)

# Get Only Protein Coding Genes

In [20]:
lst = []

for i,index in enumerate(matrix.index):
    
    progressPercent = ((i+1)/len(matrix.index)*100)

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(matrix.index)))
    sys.stdout.flush()
                                 
    lst.append(index.split('|')[7])    
                       
matrix['Gene Type'] = lst

matrix = matrix[matrix['Gene Type'] == 'protein_coding']

matrix.drop('Gene Type', axis=1, inplace=True)

In [21]:
matrix.head()

,fd85644a-09d9-4b61-993a-2da90f7907b0,56ff6499-0cc1-4108-aeb5-68ad6e371df3,bd7f0212-7a22-4b76-b5ed-f7879bad4c6b,51b4f189-f471-46c8-9a83-45dea4ff3348,91b61c25-e966-4c03-a71b-9be9e39f8465,bc939517-e335-47ca-82f9-98c580f302be,9e74ed10-91aa-44b7-9d04-7acb5d40fd85,0556b68e-a097-4d6f-b337-0554293ffab4,a1fa96bb-dd0e-4d8d-8305-f7f8c342470c,1baad8de-4a9d-4381-8970-02063c295ccc,...,e8967855-1a73-4bf0-9751-5f24d5118173,4b77ade4-dc95-4ac0-bbb3-0923e9cf625c,57d91a5a-589c-49c0-a169-8cbca345c58a,5ca8002d-74d6-42ac-805a-977c842e7740,5f8ba9ee-aafb-4902-b8d5-94d2739eb8fd,ee76b3b0-c6a8-4176-ae7a-7af338cc8ede,d541ac75-ec19-4f69-ba1d-2c28e86e0f49,4726a0f4-1c9e-4978-a4b1-47f822a4df71,ca045c46-1640-4842-9a72-4f847b413907,b46beddd-ea86-480b-a65c-aac06e3568f2
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,145.07500,285.49300,257.100000,114.16200,153.84400,121.991000,166.685000,319.796000,66.73140,207.013000,...,196.522000,117.401000,180.641000,163.23800,234.59100,316.28100,2.138910e+02,108.430000,197.75000,181.38100
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,53.03300,15.87720,14.433500,52.91730,38.91390,52.214600,83.282300,23.645200,44.19540,18.276900,...,29.718100,36.444400,38.519500,24.75210,7.72247,29.86650,1.210080e+02,5.964770,38.61060,11.49890
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,27.30070,15.12670,17.462500,21.00280,12.56990,26.143900,12.155300,9.955220,28.20380,19.578400,...,14.399500,32.773700,9.270470,13.71660,12.12620,21.86140,5.702150e+00,20.502800,15.06870,13.32210
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,80.82990,51.13760,28.742100,127.71100,44.92330,51.669400,39.728500,66.014900,102.14200,42.745300,...,21.251500,105.298000,44.568900,45.29140,19.03680,49.97300,2.522550e+02,9.587440,27.71250,31.32520
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,1.34305,1.08943,0.110161,1.25061,3.21387,0.383467,0.340106,0.214055,2.04426,0.091373,...,0.737908,0.395834,0.394323,3.66791,0.17144,1.10758,3.469740e-07,0.317372,1.97814,1.22536


In [22]:
matrix.shape

(79930, 473)

# Map Sample Meta Data to Sample ID

In [23]:
getSampleID(matrix, meta)

In [24]:
matrix.head()

,TCGA-EB-A4IQ-01,TCGA-FS-A4F5-06,TCGA-EB-A4P0-01,TCGA-EB-A44R-06,TCGA-FS-A4F8-06,TCGA-FS-A4FB-06,TCGA-EB-A44Q-06,TCGA-FS-A4F4-06,TCGA-FS-A4FD-06,TCGA-EB-A4OY-01,...,TCGA-D3-A8GD-06,TCGA-D3-A2JE-06,TCGA-Z2-A8RT-06,TCGA-YD-A89C-06,TCGA-WE-A8ZN-06,TCGA-WE-A8K5-06,TCGA-GN-A8LL-06,TCGA-D3-A8GB-06,TCGA-W3-A828-06,TCGA-ER-A2NB-01
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,145.07500,285.49300,257.100000,114.16200,153.84400,121.991000,166.685000,319.796000,66.73140,207.013000,...,196.522000,117.401000,180.641000,163.23800,234.59100,316.28100,2.138910e+02,108.430000,197.75000,181.38100
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,53.03300,15.87720,14.433500,52.91730,38.91390,52.214600,83.282300,23.645200,44.19540,18.276900,...,29.718100,36.444400,38.519500,24.75210,7.72247,29.86650,1.210080e+02,5.964770,38.61060,11.49890
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,27.30070,15.12670,17.462500,21.00280,12.56990,26.143900,12.155300,9.955220,28.20380,19.578400,...,14.399500,32.773700,9.270470,13.71660,12.12620,21.86140,5.702150e+00,20.502800,15.06870,13.32210
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,80.82990,51.13760,28.742100,127.71100,44.92330,51.669400,39.728500,66.014900,102.14200,42.745300,...,21.251500,105.298000,44.568900,45.29140,19.03680,49.97300,2.522550e+02,9.587440,27.71250,31.32520
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,1.34305,1.08943,0.110161,1.25061,3.21387,0.383467,0.340106,0.214055,2.04426,0.091373,...,0.737908,0.395834,0.394323,3.66791,0.17144,1.10758,3.469740e-07,0.317372,1.97814,1.22536


# Map Gene Symbol to Gene ID

In [25]:
getGenes(matrix)

In [26]:
matrix.head()

,TCGA-EB-A4IQ-01,TCGA-FS-A4F5-06,TCGA-EB-A4P0-01,TCGA-EB-A44R-06,TCGA-FS-A4F8-06,TCGA-FS-A4FB-06,TCGA-EB-A44Q-06,TCGA-FS-A4F4-06,TCGA-FS-A4FD-06,TCGA-EB-A4OY-01,...,TCGA-D3-A8GD-06,TCGA-D3-A2JE-06,TCGA-Z2-A8RT-06,TCGA-YD-A89C-06,TCGA-WE-A8ZN-06,TCGA-WE-A8K5-06,TCGA-GN-A8LL-06,TCGA-D3-A8GB-06,TCGA-W3-A828-06,TCGA-ER-A2NB-01
ARF5,145.07500,285.49300,257.100000,114.16200,153.84400,121.991000,166.685000,319.796000,66.73140,207.013000,...,196.522000,117.401000,180.641000,163.23800,234.59100,316.28100,2.138910e+02,108.430000,197.75000,181.38100
M6PR,53.03300,15.87720,14.433500,52.91730,38.91390,52.214600,83.282300,23.645200,44.19540,18.276900,...,29.718100,36.444400,38.519500,24.75210,7.72247,29.86650,1.210080e+02,5.964770,38.61060,11.49890
ESRRA,27.30070,15.12670,17.462500,21.00280,12.56990,26.143900,12.155300,9.955220,28.20380,19.578400,...,14.399500,32.773700,9.270470,13.71660,12.12620,21.86140,5.702150e+00,20.502800,15.06870,13.32210
FKBP4,80.82990,51.13760,28.742100,127.71100,44.92330,51.669400,39.728500,66.014900,102.14200,42.745300,...,21.251500,105.298000,44.568900,45.29140,19.03680,49.97300,2.522550e+02,9.587440,27.71250,31.32520
CYP26B1,1.34305,1.08943,0.110161,1.25061,3.21387,0.383467,0.340106,0.214055,2.04426,0.091373,...,0.737908,0.395834,0.394323,3.66791,0.17144,1.10758,3.469740e-07,0.317372,1.97814,1.22536


# Histogram of First Sample

In [27]:
sns.distplot(matrix.iloc[:, 0]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Histogram of First Gene

In [28]:
sns.distplot(matrix.iloc[0, :]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Save Unfiltered Matrix to File

In [29]:
filename = path+'tcga_skcm_matrix_unfilltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
matrix.to_csv(filename, sep='\t', compression='gzip')

# Filtered

In [30]:
normalized_matrix = matrix.copy()

In [31]:
normalized_matrix = normalized_matrix.groupby(level=0).max()

# Remove any data with more than 95% missing and imput remaining missing values to matrix mean

In [32]:
normalized_matrix = uf.removeAndImpute(normalized_matrix)

In [33]:
normalized_matrix.head()

,TCGA-EB-A4IQ-01,TCGA-FS-A4F5-06,TCGA-EB-A4P0-01,TCGA-EB-A44R-06,TCGA-FS-A4F8-06,TCGA-FS-A4FB-06,TCGA-EB-A44Q-06,TCGA-FS-A4F4-06,TCGA-FS-A4FD-06,TCGA-EB-A4OY-01,...,TCGA-D3-A8GD-06,TCGA-D3-A2JE-06,TCGA-Z2-A8RT-06,TCGA-YD-A89C-06,TCGA-WE-A8ZN-06,TCGA-WE-A8K5-06,TCGA-GN-A8LL-06,TCGA-D3-A8GB-06,TCGA-W3-A828-06,TCGA-ER-A2NB-01
A1BG,0.264855,0.264855,0.264855,0.264855,0.264855,0.264855,0.043082,0.264855,0.020656,0.264855,...,0.264855,4.221310,0.264855,0.264855,0.264855,0.264855,0.264855,1.701770,0.743793,0.264855
A1CF,0.015431,0.010665,0.007111,0.023005,0.006761,0.002536,0.349769,0.006065,0.005160,0.003110,...,0.019803,0.201308,0.013356,0.018709,0.005777,0.006913,0.009265,0.082844,0.083830,0.013865
A2M,57.947400,79.620800,40.376000,88.247900,339.200000,102.087000,81.796600,171.004000,279.924000,66.596000,...,167.064000,50.893000,266.093000,216.216000,99.509700,224.525000,336.509000,123.285000,1266.350000,27.852100
A2ML1,56.217200,0.109789,0.127098,0.091388,0.078656,0.136787,0.256243,0.052917,0.179798,0.094245,...,0.036935,0.034990,0.082031,0.059469,0.029492,0.047534,0.107916,0.020434,0.041191,37.033100
A3GALT2,0.090979,0.098742,0.072595,0.046605,0.020322,0.208666,0.092752,0.024252,0.072595,0.072595,...,0.073988,0.072595,0.140025,0.072595,0.072595,0.060931,0.072595,0.072595,0.066137,0.072595


In [34]:
normalized_matrix.shape

(19132, 473)

# logCPM Transform

Raw counts were normalized to log10-Counts Per Million (logCPM) by dividing each column by the total sum of its counts, multiplying it by 106, followed by the application of a log10-transform.

In [35]:
samples = normalized_matrix.columns.tolist()

In [36]:
normalized_matrix.columns = np.arange(0, len(normalized_matrix.columns))

In [37]:
for col in normalized_matrix.columns:

    total = normalized_matrix[col].sum()
    
    normalized_matrix[col] = normalized_matrix[col].apply(lambda x: np.log10(((x)/total)*10**6))

In [38]:
normalized_matrix.columns = samples

In [39]:
normalized_matrix.head()

,TCGA-EB-A4IQ-01,TCGA-FS-A4F5-06,TCGA-EB-A4P0-01,TCGA-EB-A44R-06,TCGA-FS-A4F8-06,TCGA-FS-A4FB-06,TCGA-EB-A44Q-06,TCGA-FS-A4F4-06,TCGA-FS-A4FD-06,TCGA-EB-A4OY-01,...,TCGA-D3-A8GD-06,TCGA-D3-A2JE-06,TCGA-Z2-A8RT-06,TCGA-YD-A89C-06,TCGA-WE-A8ZN-06,TCGA-WE-A8K5-06,TCGA-GN-A8LL-06,TCGA-D3-A8GB-06,TCGA-W3-A828-06,TCGA-ER-A2NB-01
A1BG,-0.422792,-0.397234,-0.380093,-0.357854,-0.372449,-0.389614,-1.162440,-0.443688,-1.501401,-0.414372,...,-0.313386,0.813573,-0.369206,-0.387044,-0.401611,-0.384543,-0.363763,0.432887,0.052197,-0.448647
A1CF,-1.657395,-1.792303,-1.951163,-1.419035,-1.965448,-2.408468,-0.252949,-2.083840,-2.103842,-2.344666,...,-1.439673,-0.508014,-1.666551,-1.538012,-2.062894,-1.967863,-1.819908,-0.879753,-0.895856,-1.729725
A2M,1.917234,2.080784,1.803022,2.164842,2.734998,2.196348,2.116005,2.366310,2.630584,1.986068,...,2.486488,1.894783,2.632819,2.524835,2.173246,2.543713,2.740225,2.292896,3.283298,1.573203
A2ML1,1.904069,-0.779684,-0.698963,-0.819972,-0.899728,-0.676578,-0.388078,-1.143098,-0.561671,-0.863122,...,-1.168954,-1.267931,-0.878236,-1.035759,-1.354914,-1.130546,-0.753686,-1.487671,-1.204449,1.696935
A3GALT2,-0.886860,-0.825743,-0.942193,-1.112433,-1.487493,-0.493171,-0.829408,-1.481940,-0.955548,-0.976472,...,-0.867233,-0.950967,-0.646009,-0.949145,-0.963711,-1.022714,-0.925864,-0.937106,-0.998813,-1.010747


# Normalize Matrix (Quantile Normalize the matrix for the columns)

In [40]:
normalized_matrix = uf.quantileNormalize(normalized_matrix)

In [41]:
normalized_matrix.head()

,TCGA-EB-A4IQ-01,TCGA-FS-A4F5-06,TCGA-EB-A4P0-01,TCGA-EB-A44R-06,TCGA-FS-A4F8-06,TCGA-FS-A4FB-06,TCGA-EB-A44Q-06,TCGA-FS-A4F4-06,TCGA-FS-A4FD-06,TCGA-EB-A4OY-01,...,TCGA-D3-A8GD-06,TCGA-D3-A2JE-06,TCGA-Z2-A8RT-06,TCGA-YD-A89C-06,TCGA-WE-A8ZN-06,TCGA-WE-A8K5-06,TCGA-GN-A8LL-06,TCGA-D3-A8GB-06,TCGA-W3-A828-06,TCGA-ER-A2NB-01
A1BG,-0.472567,-0.352850,-0.273768,-0.586881,-0.478017,-0.373302,-1.205875,-0.288171,-1.520733,-0.016341,...,-0.480226,0.794681,-0.445766,-0.225397,-0.147042,-0.330690,-0.211741,0.604031,-0.333230,-0.073261
A1CF,-1.682091,-1.796724,-1.904914,-1.553407,-2.012970,-2.515989,-0.319600,-2.015217,-2.178076,-2.147175,...,-1.580652,-0.352850,-1.711164,-1.458514,-1.959608,-1.925477,-1.862501,-0.864234,-1.130438,-1.590696
A2M,1.940028,2.069662,1.873058,2.117570,2.870054,2.196730,2.080170,2.482565,2.763873,2.189359,...,2.443062,1.848402,2.642712,2.580685,2.342718,2.508145,2.665524,2.486216,3.344879,1.884061
A2ML1,1.925935,-0.755754,-0.620997,-1.001927,-0.954330,-0.664294,-0.444037,-1.057715,-0.581635,-0.623453,...,-1.297101,-1.191456,-0.941735,-0.933259,-1.242952,-1.097923,-0.683557,-1.457538,-1.396592,1.973877
A3GALT2,-0.916668,-0.804571,-0.887317,-1.263436,-1.530065,-0.476034,-0.867777,-1.417717,-0.968164,-0.766533,...,-0.994009,-0.841795,-0.720188,-0.837115,-0.821161,-0.991954,-0.880505,-0.921575,-1.214478,-0.855674


# Normalize Matrix (z-score the rows)

In [42]:
uf.zscore(normalized_matrix, 'row')

In [43]:
normalized_matrix.head()

,TCGA-EB-A4IQ-01,TCGA-FS-A4F5-06,TCGA-EB-A4P0-01,TCGA-EB-A44R-06,TCGA-FS-A4F8-06,TCGA-FS-A4FB-06,TCGA-EB-A44Q-06,TCGA-FS-A4F4-06,TCGA-FS-A4FD-06,TCGA-EB-A4OY-01,...,TCGA-D3-A8GD-06,TCGA-D3-A2JE-06,TCGA-Z2-A8RT-06,TCGA-YD-A89C-06,TCGA-WE-A8ZN-06,TCGA-WE-A8K5-06,TCGA-GN-A8LL-06,TCGA-D3-A8GB-06,TCGA-W3-A828-06,TCGA-ER-A2NB-01
A1BG,-0.578514,0.309854,0.896685,-1.426783,-0.618949,0.158090,-6.020060,0.789813,-8.356480,2.806934,...,-0.635342,8.825165,-0.379631,1.255626,1.837061,0.474292,1.356961,7.410436,0.455450,2.384556
A1CF,0.360389,-0.044085,-0.425823,0.814441,-0.807090,-2.581955,5.167831,-0.815021,-1.389654,-1.280622,...,0.718310,5.050511,0.257806,1.149264,-0.618806,-0.498378,-0.276175,3.246134,2.306853,0.682871
A2M,-1.499133,-1.231264,-1.637517,-1.132268,0.422631,-0.968695,-1.209550,-0.378059,0.203224,-0.983926,...,-0.459685,-1.688466,-0.047138,-0.175307,-0.667033,-0.325202,0.000000,-0.370515,1.403788,-1.614781
A2ML1,8.385110,0.303290,0.709408,-0.438600,-0.295158,0.578925,1.242714,-0.606731,0.828033,0.702007,...,-1.328170,-1.009786,-0.257201,-0.231658,-1.164979,-0.727906,0.520871,-1.811680,-1.628006,8.529595
A3GALT2,0.278867,0.774986,0.408769,-1.255869,-2.435922,2.229036,0.495248,-1.938692,0.050954,0.943337,...,-0.063431,0.610240,1.148454,0.630952,0.701562,-0.054336,0.438916,0.257147,-1.039192,0.548812


# Merge Like Column (by taking the mean)

In [44]:
normalized_matrix = uf.merge(normalized_matrix, 'column', 'mean')

In [45]:
normalized_matrix.shape

(19132, 473)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [46]:
uf.mapgenesymbols(normalized_matrix)

In [47]:
normalized_matrix.shape

# Merge Duplicate Genes By Rows

In [48]:
normalized_matrix = uf.merge(normalized_matrix, 'row', 'mean')

In [49]:
normalized_matrix.shape

(18612, 473)

# Histogram of First Sample

In [50]:
sns.distplot(normalized_matrix.iloc[:, 10]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Histogram of First Gene

In [51]:
sns.distplot(normalized_matrix.iloc[10, :]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Save Filtered Matrix

In [52]:
filename = path+'tcga_skcm_matrix_filltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
normalized_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [53]:
gene_list = uf.createGeneList(normalized_matrix)

In [54]:
gene_list.head()

,GeneSym,GeneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A3GALT2,127550


In [55]:
gene_list.shape

(18612, 2)

# Save Gene List

In [56]:
filename = path+'tcga_skcm_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute List

In [57]:
metaData = pd.DataFrame(index=normalized_matrix.columns)

for index in metaData.index:
    if ('-').join(index.split('-')[0:3]) in sample_meta.index:
        metaData.loc[index, 'Age'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'age_at_initial_pathologic_diagnosis']
        metaData.loc[index, 'Gender'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'gender']
        metaData.loc[index, 'Pathologic M'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_M']
        metaData.loc[index, 'Pathologic N'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_N']
        metaData.loc[index, 'Pathologic T'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_T']
        metaData.loc[index, 'Pathologic Stage'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_stage']
        metaData.loc[index, 'Histological Type'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'histological_type']
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'tumor_tissue_site']
        metaData.loc[index, 'Neoplasm Histologic Grade'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'neoplasm_histologic_grade']
    else:
        metaData.loc[index, 'Age'] = np.nan
        metaData.loc[index, 'Gender'] = np.nan
        metaData.loc[index, 'Pathologic M'] = np.nan
        metaData.loc[index, 'Pathologic N'] = np.nan
        metaData.loc[index, 'Pathologic T'] = np.nan
        metaData.loc[index, 'Pathologic Stage'] = np.nan
        metaData.loc[index, 'Histological Type'] = np.nan
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = np.nan
        metaData.loc[index, 'Neoplasm Histologic Grade'] = np.nan

In [58]:
attribute_list = uf.createAttributeList(normalized_matrix, metaData)

In [59]:
attribute_list.head()

,Age,Gender,Pathologic M,Pathologic N,Pathologic T,Pathologic Stage,Histological Type,Sample Type,Tumor Tissue Site,Neoplasm Histologic Grade
Attributes,,,,,,,,,,
TCGA-3N-A9WB-06,71,MALE,M0,NX,T1a,Stage IA,Melanoma,Metastatic,Trunk,NaN
TCGA-3N-A9WC-06,82,MALE,M0,NX,T2b,Stage IIA,Melanoma,Metastatic,Trunk,NaN
TCGA-3N-A9WD-06,82,MALE,M0,N1a,T2a,Stage IIIA,Melanoma,Metastatic,Other Specify,NaN
TCGA-BF-A1PU-01,46,FEMALE,M0,N0,T4b,Stage IIC,Melanoma,Primary Solid Tumor,Extremities,NaN
TCGA-BF-A1PV-01,74,FEMALE,M0,N0,T4b,Stage IIC,Melanoma,Primary Solid Tumor,Trunk,NaN


In [60]:
attribute_list.shape

(473, 10)

# Save Attribute List

In [61]:
filename = path+'tcga_skcm_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', compression='gzip')

# Create matrix of Standardized values (values between -1, and 1)

In [62]:
standard_matrix = uf.createStandardizedMatrix(normalized_matrix)

In [63]:
standard_matrix.head()

,TCGA-3N-A9WB-06,TCGA-3N-A9WC-06,TCGA-3N-A9WD-06,TCGA-BF-A1PU-01,TCGA-BF-A1PV-01,TCGA-BF-A1PX-01,TCGA-BF-A1PZ-01,TCGA-BF-A1Q0-01,TCGA-BF-A3DJ-01,TCGA-BF-A3DL-01,...,TCGA-XV-AB01-06,TCGA-YD-A89C-06,TCGA-YD-A9TA-06,TCGA-YD-A9TB-06,TCGA-YG-AA3N-01,TCGA-YG-AA3O-06,TCGA-YG-AA3P-06,TCGA-Z2-A8RT-06,TCGA-Z2-AA3S-06,TCGA-Z2-AA3V-06
,,,,,,,,,,,,,,,,,,,,,
A1BG,0.640910,-0.225954,0.065663,0.543501,-0.881077,-0.116058,0.581590,-0.327419,-0.986640,0.924533,...,0.848308,0.755206,-0.766919,-0.859934,0.509686,-0.893745,0.865271,-0.331664,-0.762704,0.903368
A1CF,-0.666069,0.449897,-0.323775,0.572566,-0.294159,-0.898539,0.352641,0.479501,-0.116639,-0.361791,...,0.086280,0.835040,0.902800,-0.649162,-0.877421,-0.615367,0.145487,0.217292,0.213097,-0.759051
A2M,-0.974573,0.974751,-0.234999,-0.729484,-0.978794,0.060874,-0.945043,-0.750619,0.987436,-0.746379,...,-0.391363,-0.116649,-0.344879,0.183478,-0.535060,0.458346,-0.598462,-0.015249,-0.847831,0.191946
A2ML1,0.406147,-0.156132,-0.828130,0.494974,-0.845046,0.956381,0.655810,0.376542,0.888657,-0.308293,...,-0.980227,-0.177317,-0.257557,-0.663275,0.935209,0.338504,0.258146,-0.227982,0.055163,0.825138
A3GALT2,0.429388,-0.373791,0.124992,0.442096,0.319489,-0.999113,-0.847145,-0.204749,0.895014,-0.593519,...,0.217967,0.649499,-0.132870,-0.107527,0.302561,0.298331,0.670610,0.852670,0.268756,0.535162


# Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [64]:
plt.plot(normalized_matrix[normalized_matrix.columns[0]], standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

# Save Standardized Matrix

In [65]:
filename = path+'tcga_skcm_standard_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
standard_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Tertiary Matrix

In [66]:
tertiary_matrix = uf.createTertiaryMatrix(standard_matrix)

In [67]:
tertiary_matrix.head()

,TCGA-3N-A9WB-06,TCGA-3N-A9WC-06,TCGA-3N-A9WD-06,TCGA-BF-A1PU-01,TCGA-BF-A1PV-01,TCGA-BF-A1PX-01,TCGA-BF-A1PZ-01,TCGA-BF-A1Q0-01,TCGA-BF-A3DJ-01,TCGA-BF-A3DL-01,...,TCGA-XV-AB01-06,TCGA-YD-A89C-06,TCGA-YD-A9TA-06,TCGA-YD-A9TB-06,TCGA-YG-AA3N-01,TCGA-YG-AA3O-06,TCGA-YG-AA3P-06,TCGA-Z2-A8RT-06,TCGA-Z2-AA3S-06,TCGA-Z2-AA3V-06
,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,0,0,0,0,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,-1,1,0,0,-1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
A2ML1,0,0,0,0,0,1,0,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
A3GALT2,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Save Teriary Matrix

In [68]:
filename = path+'tcga_skcm_tertiary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
tertiary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Up Gene Set Library

In [69]:
name = 'tcga_skcm_gene_up_set'

In [70]:
uf.createUpGeneSetLib(tertiary_matrix, path, name)

# Create Down Gene Set Library

In [71]:
name = 'tcga_skcm_gene_down_set'

In [72]:
uf.createDownGeneSetLib(tertiary_matrix, path, name)

# Create Up Attribute Library

In [73]:
name = 'tcga_skcm_attribute_up_set'

In [74]:
uf.createUpAttributeSetLib(tertiary_matrix, path, name)

# Create Down Attribute Library

In [75]:
name = 'tcga_skcm_attribute_down_set'

In [76]:
uf.createDownAttributeSetLib(tertiary_matrix, path, name)

# Create Attribute Similarity Matrix

In [77]:
attribute_normalized_matrix = normalized_matrix.copy()

In [78]:
getSampleData(attribute_normalized_matrix, sample_meta, SampleTypes)

In [79]:
attribute_similarity_matix = uf.createSimilarityMatrix(attribute_normalized_matrix.T, 'cosine')

In [80]:
attribute_similarity_matix.head()

,"(Donor:TCGA-3N-A9WB, Age:71.0, Gender:MALE, SampleType:Metastatic, Stage:Stage IA, Histology:Melanoma, Tissue:Trunk, Tumor:nan, Gradenan)","(Donor:TCGA-3N-A9WC, Age:82.0, Gender:MALE, SampleType:Metastatic, Stage:Stage IIA, Histology:Melanoma, Tissue:Trunk, Tumor:nan, Gradenan)","(Donor:TCGA-3N-A9WD, Age:82.0, Gender:MALE, SampleType:Metastatic, Stage:Stage IIIA, Histology:Melanoma, Tissue:Other Specify, Tumor:nan, Gradenan)","(Donor:TCGA-BF-A1PU, Age:46.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IIC, Histology:Melanoma, Tissue:Extremities, Tumor:nan, Gradenan)","(Donor:TCGA-BF-A1PV, Age:74.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IIC, Histology:Melanoma, Tissue:Trunk, Tumor:nan, Gradenan)","(Donor:TCGA-BF-A1PX, Age:56.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIIB, Histology:Melanoma, Tissue:Trunk, Tumor:nan, Gradenan)","(Donor:TCGA-BF-A1PZ, Age:71.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IIB, Histology:Melanoma, Tissue:Extremities, Tumor:nan, Gradenan)","(Donor:TCGA-BF-A1Q0, Age:80.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIC, Histology:Melanoma, Tissue:Trunk, Tumor:nan, Gradenan)","(Donor:TCGA-BF-A3DJ, Age:36.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IIIB, Histology:Melanoma, Tissue:Extremities, Tumor:nan, Gradenan)","(Donor:TCGA-BF-A3DL, Age:84.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IIIB, Histology:Melanoma, Tissue:Extremities, Tumor:nan, Gradenan)",...,"(Donor:TCGA-XV-AB01, Age:54.0, Gender:FEMALE, SampleType:Metastatic, Stage:Stage II, Histology:Melanoma, Tissue:Head and Neck, Tumor:nan, Gradenan)","(Donor:TCGA-YD-A89C, Age:43.0, Gender:FEMALE, SampleType:Metastatic, Stage:Stage IA, Histology:Melanoma, Tissue:Extremities, Tumor:nan, Gradenan)","(Donor:TCGA-YD-A9TA, Age:75.0, Gender:MALE, SampleType:Metastatic, Stage:nan, Histology:Melanoma, Tissue:Head and Neck, Tumor:nan, Gradenan)","(Donor:TCGA-YD-A9TB, Age:nan, Gender:FEMALE, SampleType:Metastatic, Stage:nan, Histology:Melanoma, Tissue:Head and Neck, Tumor:nan, Gradenan)","(Donor:TCGA-YG-AA3N, Age:67.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIC, Histology:Melanoma, Tissue:Trunk, Tumor:nan, Gradenan)","(Donor:TCGA-YG-AA3O, Age:62.0, Gender:MALE, SampleType:Metastatic, Stage:nan, Histology:Melanoma, Tissue:Trunk, Tumor:nan, Gradenan)","(Donor:TCGA-YG-AA3P, Age:63.0, Gender:FEMALE, SampleType:Metastatic, Stage:Stage IIB, Histology:Melanoma, Tissue:Extremities, Tumor:nan, Gradenan)","(Donor:TCGA-Z2-A8RT, Age:42.0, Gender:FEMALE, SampleType:Metastatic, Stage:Stage IIB, Histology:Melanoma, Tissue:Extremities, Tumor:nan, Gradenan)","(Donor:TCGA-Z2-AA3S, Age:58.0, Gender:MALE, SampleType:Metastatic, Stage:Stage IA, Histology:Melanoma, Tissue:Trunk, Tumor:nan, Gradenan)","(Donor:TCGA-Z2-AA3V, Age:57.0, Gender:FEMALE, SampleType:Metastatic, Stage:Stage IA, Histology:Melanoma, Tissue:Trunk, Tumor:nan, Gradenan)"
,,,,,,,,,,,,,,,,,,,,,
"(Donor:TCGA-3N-A9WB, Age:71.0, Gender:MALE, SampleType:Metastatic, Stage:Stage IA, Histology:Melanoma, Tissue:Trunk, Tumor:nan, Gradenan)",1.000000,0.138700,-0.026011,0.098875,0.071497,0.156817,0.158501,0.074857,0.038166,0.026772,...,-0.041887,0.022882,0.023184,0.016431,0.049832,0.058583,0.025225,0.019600,-0.005555,0.009889
"(Donor:TCGA-3N-A9WC, Age:82.0, Gender:MALE, SampleType:Metastatic, Stage:Stage IIA, Histology:Melanoma, Tissue:Trunk, Tumor:nan, Gradenan)",0.138700,1.000000,0.031937,-0.022546,0.055765,0.006816,-0.015279,0.185706,0.122438,-0.063924,...,-0.061376,-0.052643,0.147103,0.064799,0.067170,0.033101,0.004210,0.087580,0.018338,-0.051691
"(Donor:TCGA-3N-A9WD, Age:82.0, Gender:MALE, SampleType:Metastatic, Stage:Stage IIIA, Histology:Melanoma, Tissue:Other Specify, Tumor:nan, Gradenan)",-0.026011,0.031937,1.000000,0.135193,-0.036821,0.108651,-0.008801,0.033554,0.058001,-0.020657,...,0.046513,0.005441,0.045227,0.064403,-0.037908,-0.017866,0.013474,0.031768,-0.026946,0.012235
"(Dono

# Save Attribute Similarity Matrix

In [81]:
filename = path+'tcga_skcm_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# HeatMap (clustergrammer) of Similarity Matrix

In [82]:
net.load_df(attribute_similarity_matix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
net.cluster()
net.widget()

clustergrammer_widget(network='{"views": [{"dist": "cos", "nodes": {"col_nodes": [{"cat_4_index": 0, "cat_0_in…

# Create Gene Similarity Matrix

In [83]:
gene_similarity_matix = uf.createSimilarityMatrix(normalized_matrix, 'cosine')

In [84]:
gene_similarity_matix.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,0.024303,-0.018217,-0.157252,0.057643,-0.044185,0.096549,0.110917,-0.073359,0.207385,...,-0.137280,0.054730,-0.078059,-0.043853,0.062434,-0.118184,-0.123528,-0.021831,0.031315,-0.110704
A1CF,0.024303,1.000000,-0.109656,-0.045758,0.070992,-0.132119,0.019604,-0.010343,-0.042747,-0.036563,...,0.067535,0.003630,-0.032420,0.001147,-0.031400,0.074727,0.045497,-0.096714,-0.024098,0.086190
A2M,-0.018217,-0.109656,1.000000,-0.162503,-0.013471,0.067355,0.075704,-0.251226,-0.100243,0.016711,...,-0.085901,-0.266517,0.143278,0.047295,-0.048466,-0.050682,0.157341,0.111677,-0.031928,0.133082
A2ML1,-0.157252,-0.045758,-0.162503,1.000000,-0.090504,0.047354,-0.125068,-0.044782,0.223390,-0.180069,...,-0.175480,0.003141,-0.135585,-0.122967,0.038531,-0.002091,-0.141500,-0.051636,0.098124,-0.178967
A3GALT2,0.057643,0.070992,-0.013471,-0.090504,1.000000,-0.019564,0.117873,0.100955,-0.018350,0.134746,...,-0.011111,0.059222,-0.069669,-0.075551,0.046055,0.042698,0.001686,-0.021791,0.048872,-0.041083


# Save Gene Similarity Matrix

In [85]:
filename = path+'tcga_skcm_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene-Attribute Edge List

In [86]:
name = 'tcga_skcm_gene_attribute_edge_list'

In [87]:
uf.createGeneAttributeEdgeList(standard_matrix, attribute_list, gene_list, path, name)

Progeres: 100%  473 Out of 473   

 The number of statisticaly relevent gene-attribute associations is: 439949
